In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pathlib import Path
from torch.optim.lr_scheduler import _LRScheduler

In [34]:


class TimeSeriesDataset(Dataset):
    def __init__(self, x_set, y_set, seq_len):
        self.x, self.y = x_set, y_set
        self.seq_len = seq_len

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.seq_len)))


    def __getitem__(self, idx):
        start_idx = idx * self.seq_len
        end_idx = (idx + 1) * self.seq_len

        batch_x = self.x[start_idx:end_idx]
        batch_y = self.y[start_idx:end_idx]

        # Count the occurrences of each row
        unique_rows, counts = np.unique(batch_y, axis=0, return_counts=True)

        # Get the index of the row with the highest count
        most_common_row_index = np.argmax(counts)

        # Get the most common row
        most_common_row = unique_rows[most_common_row_index]
        
        
        batch_y_bin = most_common_row

        # Convert NumPy arrays to PyTorch tensors
        batch_x = torch.from_numpy(batch_x)
        batch_y = torch.from_numpy(batch_y)
        batch_y_bin = torch.from_numpy(batch_y_bin)

        # Pad sequences to ensure they have the same length within the batch
        pad_len = self.seq_len - batch_x.shape[0]
        if pad_len > 0:
            pad_shape = (pad_len,) + batch_x.shape[1:]
            pad_shape_y = (pad_len,) + batch_y.shape[1:]

            batch_x = torch.cat([batch_x, torch.zeros(pad_shape)], dim=0)
            batch_y = torch.cat([batch_y, torch.zeros(pad_shape_y)], dim=0)

        return batch_x, batch_y, batch_y_bin

    def on_epoch_end(self):
        indices = np.arange(len(self.x))
        np.random.shuffle(indices)
        self.x = self.x[indices]
        self.y = self.y[indices]


In [35]:
def generate_data(subject_id, task, features, batch_size, seq_len):    
    
    csv_path = './ProcessedDatasets/' + task
    # csv_path = './Dataset'
    
    csv_files = glob.glob(csv_path + "/*.csv")
    
    train_df_list = []
    test_df_list = []
    
    for file in csv_files:
        if(subject_id in file):
            test_df_list.append(pd.read_csv(file))
#             print(file)
        else:
            train_df_list.append(pd.read_csv(file))
            

    print('Train Subject Trials: ',len(train_df_list))
    print('Test Subject Trials: ',len(test_df_list))
    
    # Concatenate all DataFrames
    train_df   = pd.concat(train_df_list, ignore_index=True)
    test_df   = pd.concat(test_df_list, ignore_index=True)

    
    lb = preprocessing.LabelBinarizer()

    train_labels= train_df.pop('label')
    train_features = train_df

    test_labels= test_df.pop('label')
    test_features = test_df


    all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']
    lb.fit(all_class_names)

    train_labels = lb.transform(train_labels)
    test_labels = lb.transform(test_labels)
    
    train_x = train_features.to_numpy()
    train_y = train_labels

    test_x = test_features.to_numpy()
    test_y = test_labels
    
    train_x = train_x[:,:features]
    test_x = test_x[:,:features]
    

    train_dataset = TimeSeriesDataset(train_x, train_y, seq_len)
    test_dataset = TimeSeriesDataset(test_x, test_y, seq_len)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    return train_dataloader, test_dataloader
 
    
    
features = 10
batch_size = 30
seq_len = 30
output_dim = 14

train_dataloader, test_dataloader = generate_data("S02","Knot_Tying",features, batch_size, seq_len)

for idx,batch in enumerate(train_dataloader):
    
    print(idx, batch[0].shape ,batch[1].shape ,batch[2].shape)
    break

Train Subject Trials:  32
Test Subject Trials:  4
0 torch.Size([30, 30, 10]) torch.Size([30, 30, 14]) torch.Size([30, 14])


In [36]:
print(torch.__version__)

2.0.1+cu117


In [54]:



def train_loop(dataloader,model,optimizer,scheduler,criterion, epochs):
    
    # training loop
    for epoch in range(epochs):
        running_loss = 0.0
        for i, batch in enumerate(dataloader):

            
            x, y, y_seq = batch
            x = x.to(torch.float32)
            y = y.to(torch.float32)
            y_seq = y_seq.to(torch.float32)
            
            x = x.cuda()
            y = y.cuda()
            y_seq = y_seq.cuda()
            
            scheduler.step()
            optimizer.zero_grad()
            
            y_pred = model(x)

            # print(y_pred.shape, y_seq.shape)
            
            loss = criterion(y_pred, y_seq)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader):.6f}")


In [37]:
def evaluation_loop(dataloader, model,criterion):

    model.eval()

    total_accuracy = []
    for i, batch in enumerate(test_dataloader):
        x, y, y_seq = batch
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        y_seq = y_seq.to(torch.float32)
        
        x = x.cuda()
        y = y.cuda()
        y_seq = y_seq.cuda()
        
        y_pred = model(x)
        
        total_inputs = 0
        true_pred = []
        
        
        for idx,y in enumerate(y_pred):
            
            total_inputs += 1
            
            output_argmax = torch.argmax(y)
            gt_argmax = torch.argmax(y_seq[idx])
        
            if(output_argmax == gt_argmax):
                true_pred.append(output_argmax)
                
            accuracy = len(true_pred)/total_inputs
            
            # print("Accuracy: ",accuracy)
            total_accuracy.append(accuracy)
            
            
                
        loss = criterion(y_pred, y_seq)
        # print(i, "Loss: ", loss)
        
    avg_accuracy = np.average(total_accuracy)
    print("Average accuracy: ", avg_accuracy)
    return avg_accuracy
    
    

In [38]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device=x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])

        return out

In [39]:

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]



def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



In [78]:
def initiate_model(input_dim,output_dim, hidden_dim, layer_dim, seq_dim, lr, iterations_per_epoch ):



    model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    
    
    model = model.cuda()
    
    optimizer = optim.Adam(model.parameters(), lr=lr) # adam
    
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=lr) # custom
    
    sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

    criterion = nn.CrossEntropyLoss()
    
    
    
    return model, optimizer,sched, criterion

In [ ]:
        

lr = 0.0001

features = 36
batch_size = 64
seq_len = 10

input_dim = features  
output_dim = 14

hidden_dim = 256
layer_dim = 3
seq_dim = 128

task = "Knot_Tying"

epochs = 500
iterations_per_epoch = len(train_dataloader)

model,optimizer,scheduler,criterion = initiate_model(input_dim,output_dim, hidden_dim, layer_dim, seq_dim, lr , iterations_per_epoch=iterations_per_epoch)
    

In [83]:
subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']
# subjects = ['S03']

accuracy = []
for subject in subjects:
    
    model,optimizer,scheduler,criterion = initiate_model(input_dim,output_dim, hidden_dim, layer_dim, seq_dim, lr , iterations_per_epoch=iterations_per_epoch)

    train_dataloader, test_dataloader = generate_data(subject,task,features, batch_size, seq_len)

    train_loop(dataloader=train_dataloader, model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, epochs=epochs)
    
    acc = evaluation_loop(dataloader=test_dataloader, model=model, criterion=criterion)
    
    subject_accuracy = {'subject':subject, 'accuracy':acc}
    print(subject_accuracy)
    accuracy.append(subject_accuracy)

    

print(accuracy)


Train Subject Trials:  32
Test Subject Trials:  4


/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1, Loss: 2.079226
Epoch 2, Loss: 1.576540
Epoch 3, Loss: 1.441577
Epoch 4, Loss: 1.300914
Epoch 5, Loss: 1.297079
Epoch 6, Loss: 1.173187
Epoch 7, Loss: 1.171864
Epoch 8, Loss: 1.039167
Epoch 9, Loss: 1.074939
Epoch 10, Loss: 0.945293
Epoch 11, Loss: 0.998160
Epoch 12, Loss: 0.877637
Epoch 13, Loss: 0.938416
Epoch 14, Loss: 0.828836
Epoch 15, Loss: 0.890666
Epoch 16, Loss: 0.790032
Epoch 17, Loss: 0.859909
Epoch 18, Loss: 0.750576
Epoch 19, Loss: 0.804539
Epoch 20, Loss: 0.715714
Epoch 21, Loss: 0.766747
Epoch 22, Loss: 0.686011
Epoch 23, Loss: 0.730998
Epoch 24, Loss: 0.658830
Epoch 25, Loss: 0.702689
Epoch 26, Loss: 0.636201
Epoch 27, Loss: 0.675750
Epoch 28, Loss: 0.609015
Epoch 29, Loss: 0.643065
Epoch 30, Loss: 0.587098
Epoch 31, Loss: 0.614496
Epoch 32, Loss: 0.565088
Epoch 33, Loss: 0.589445
Epoch 34, Loss: 0.537479
Epoch 35, Loss: 0.563176
Epoch 36, Loss: 0.516011
Epoch 37, Loss: 0.539729
Epoch 38, Loss: 0.501574
Epoch 39, Loss: 0.517936
Epoch 40, Loss: 0.480881
Epoch 41,

/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1, Loss: 2.053515
Epoch 2, Loss: 1.577042
Epoch 3, Loss: 1.459763
Epoch 4, Loss: 1.304253
Epoch 5, Loss: 1.300740
Epoch 6, Loss: 1.175779
Epoch 7, Loss: 1.193079
Epoch 8, Loss: 1.077770
Epoch 9, Loss: 1.118491
Epoch 10, Loss: 1.003525
Epoch 11, Loss: 1.050360
Epoch 12, Loss: 0.947537
Epoch 13, Loss: 0.998235
Epoch 14, Loss: 0.899907
Epoch 15, Loss: 0.932773
Epoch 16, Loss: 0.840993
Epoch 17, Loss: 0.872021
Epoch 18, Loss: 0.788828
Epoch 19, Loss: 0.852932
Epoch 20, Loss: 0.767685
Epoch 21, Loss: 0.799037
Epoch 22, Loss: 0.727480
Epoch 23, Loss: 0.765935
Epoch 24, Loss: 0.696676
Epoch 25, Loss: 0.758546
Epoch 26, Loss: 0.675387
Epoch 27, Loss: 0.707588
Epoch 28, Loss: 0.661281
Epoch 29, Loss: 0.685618
Epoch 30, Loss: 0.625722
Epoch 31, Loss: 0.649192
Epoch 32, Loss: 0.604333
Epoch 33, Loss: 0.635323
Epoch 34, Loss: 0.583640
Epoch 35, Loss: 0.621037
Epoch 36, Loss: 0.565939
Epoch 37, Loss: 0.585951
Epoch 38, Loss: 0.556863
Epoch 39, Loss: 0.584502
Epoch 40, Loss: 0.538675
Epoch 41,

...

Epoch 1, Loss: 2.070937
Epoch 2, Loss: 1.594816
Epoch 3, Loss: 1.527464
Epoch 4, Loss: 1.354185
Epoch 5, Loss: 1.245716
Epoch 6, Loss: 1.182092
Epoch 7, Loss: 1.032487
Epoch 8, Loss: 1.066735
Epoch 9, Loss: 0.937535
Epoch 10, Loss: 0.999222
Epoch 11, Loss: 0.926848
Epoch 12, Loss: 0.883103
Epoch 13, Loss: 0.894263
Epoch 14, Loss: 0.805876
Epoch 15, Loss: 0.843539
Epoch 16, Loss: 0.753764
Epoch 17, Loss: 0.856880
Epoch 18, Loss: 0.768282
Epoch 19, Loss: 0.740562
Epoch 20, Loss: 0.765460
Epoch 21, Loss: 0.699092
Epoch 22, Loss: 0.733012
Epoch 23, Loss: 0.650767
Epoch 24, Loss: 0.739633
Epoch 25, Loss: 0.654351
Epoch 26, Loss: 0.678416
Epoch 27, Loss: 0.649337
Epoch 28, Loss: 0.613248
Epoch 29, Loss: 0.645636
Epoch 30, Loss: 0.579291
Epoch 31, Loss: 0.635391
Epoch 32, Loss: 0.553444
Epoch 33, Loss: 0.639634
Epoch 34, Loss: 0.584971
Epoch 35, Loss: 0.580113
Epoch 36, Loss: 0.569242
Epoch 37, Loss: 0.538374
Epoch 38, Loss: 0.568005
Epoch 39, Loss: 0.497217
Epoch 40, Loss: 0.575787
Epoch 41,

/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1, Loss: 2.045905
Epoch 2, Loss: 1.631765
Epoch 3, Loss: 1.495801
Epoch 4, Loss: 1.364480
Epoch 5, Loss: 1.314476
Epoch 6, Loss: 1.201824
Epoch 7, Loss: 1.166679
Epoch 8, Loss: 1.063142
Epoch 9, Loss: 1.049161
Epoch 10, Loss: 0.964615
Epoch 11, Loss: 0.981620
Epoch 12, Loss: 0.928036
Epoch 13, Loss: 0.912346
Epoch 14, Loss: 0.908378
Epoch 15, Loss: 0.880028
Epoch 16, Loss: 0.881853
Epoch 17, Loss: 0.862334
Epoch 18, Loss: 0.851018
Epoch 19, Loss: 0.827377
Epoch 20, Loss: 0.834343
Epoch 21, Loss: 0.805814
Epoch 22, Loss: 0.799749
Epoch 23, Loss: 0.770871
Epoch 24, Loss: 0.804384
Epoch 25, Loss: 0.748259
Epoch 26, Loss: 0.772723
Epoch 27, Loss: 0.715763
Epoch 28, Loss: 0.763848
Epoch 29, Loss: 0.697449
Epoch 30, Loss: 0.738972
Epoch 31, Loss: 0.673557
Epoch 32, Loss: 0.706837
Epoch 33, Loss: 0.642984
Epoch 34, Loss: 0.691837
Epoch 35, Loss: 0.620705
Epoch 36, Loss: 0.679698
Epoch 37, Loss: 0.609150
Epoch 38, Loss: 0.642485
Epoch 39, Loss: 0.606531
Epoch 40, Loss: 0.623114
Epoch 41,

/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1, Loss: 2.120220
Epoch 2, Loss: 1.594027
Epoch 3, Loss: 1.472395
Epoch 4, Loss: 1.323455
Epoch 5, Loss: 1.226130
Epoch 6, Loss: 1.164606
Epoch 7, Loss: 1.077304
Epoch 8, Loss: 1.087757
Epoch 9, Loss: 0.992231
Epoch 10, Loss: 1.033243
Epoch 11, Loss: 0.924309
Epoch 12, Loss: 0.959752
Epoch 13, Loss: 0.863969
Epoch 14, Loss: 0.912730
Epoch 15, Loss: 0.814885
Epoch 16, Loss: 0.923466
Epoch 17, Loss: 0.806768
Epoch 18, Loss: 0.859316
Epoch 19, Loss: 0.767220
Epoch 20, Loss: 0.742510
Epoch 21, Loss: 0.736080
Epoch 22, Loss: 0.685913
Epoch 23, Loss: 0.708951
Epoch 24, Loss: 0.693253
Epoch 25, Loss: 0.712268
Epoch 26, Loss: 0.664962
Epoch 27, Loss: 0.676506
Epoch 28, Loss: 0.618418
Epoch 29, Loss: 0.685324
Epoch 30, Loss: 0.577240
Epoch 31, Loss: 0.643111
Epoch 32, Loss: 0.553408
Epoch 33, Loss: 0.642586
Epoch 34, Loss: 0.561389
Epoch 35, Loss: 0.584381
Epoch 36, Loss: 0.530178
Epoch 37, Loss: 0.525790
Epoch 38, Loss: 0.512487
Epoch 39, Loss: 0.479831
Epoch 40, Loss: 0.521381
Epoch 41,

In [104]:
acc = []
for x in accuracy:
    acc.append(x['accuracy'])
    
print(np.average(acc))

0.5368470202782913


In [77]:
# 2. Create model save path 
MODEL_PATH = "./checkpoints/"
MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_NAME = str(d_model) + "_" + str(nhead) + "_" + str(num_layers) + "_" + MODEL_NAME
MODEL_SAVE_PATH = MODEL_PATH +"/"+ MODEL_NAME



# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")

torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 


print("done saving!")

Saving model to: ./checkpoints//64_4_2_01_pytorch_workflow_model_0.pth
done saving!


In [94]:
def evaluation_loop():

    model.eval()

    total_accuracy = []
    for i, batch in enumerate(test_dataloader):
        x, y, y_seq = batch
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        y_seq = y_seq.to(torch.float32)
        
        y_pred = model(x)
        
        total_inputs = 0
        true_pred = []
        
        
        for idx,y in enumerate(y_pred):
            
            total_inputs += 1
            
            output_argmax = torch.argmax(y)
            gt_argmax = torch.argmax(y_seq[idx])
        
            if(output_argmax == gt_argmax):
                true_pred.append(output_argmax)
                
            accuracy = len(true_pred)/total_inputs
            
            # print("Accuracy: ",accuracy)
            total_accuracy.append(accuracy)
            
            
                
        loss = criterion(y_pred, y_seq)
        # print(i, "Loss: ", loss)
        
    print("Average accuracy: ", np.average(total_accuracy))
    
    
evaluation_loop()

Average accuracy:  0.28562257379625866
